# Minimalist Algebras Demo: MG+2.3

This Notebook will take you through some of the features of the Python code I wrote to implement my notion of Generalised Minimalist Algebras.

These implement only the structure-building component of an MG. From the two-step perspective, these are just for the second step. (To incorporate features, there need to be rules mapping an operation in a given feature state to the algebra operation. These are implemented for John Torr's MGBank grammar, but I won't talk about those today. You can find them in `minimalist_parser.convert_mgbank`.)

This code is very object-oriented with a lot of inheritance and default behaviour.

We will start with the `Algebra` class.

In [6]:
from minimalist_parser.algebras.string_algebra import BareTreeStringAlgebra
from minimalist_parser.algebras.algebra import AlgebraOp, AlgebraTerm
from minimalist_parser.minimalism.minimalist_algebra_synchronous import MinimalistAlgebraSynchronous, MinimalistFunctionSynchronous, SynchronousTerm, InnerAlgebraInstructions
from minimalist_parser.algebras.hm_algebra import HMAlgebra
from minimalist_parser.minimalism.prepare_packages.prepare_packages_bare_trees import PreparePackagesBareTrees
from minimalist_parser.minimalism.prepare_packages.prepare_packages_hm import PreparePackagesHM




We start by initialising an algebra over strings with operation names < and >. The terms over this algebra are thus Bare Trees a la Stabler 1997.

`BareTreeStringAlgebra` is a subclass of `HMAlgebra`, which I designed for head movement.

In [7]:
string_algebra = BareTreeStringAlgebra()

help(string_algebra)

Help on BareTreeStringAlgebra in module minimalist_parser.algebras.string_algebra object:

class BareTreeStringAlgebra(minimalist_parser.algebras.hm_algebra.HMAlgebra)
 |  BareTreeStringAlgebra(name=None, zero=None, ops=None, syntax_op_names=None, meta=None)
 |  
 |  A string algebra in which the concatenate operation names are > and <, and for head movement, <_h and >_h.
 |  This makes for a simple string algebra in which the algebra terms are Bare Trees a la Stabler 1997.
 |  
 |  Note that this is different from a tree-building algebra, in which the objects built are actually trees.
 |  
 |  Method resolution order:
 |      BareTreeStringAlgebra
 |      minimalist_parser.algebras.hm_algebra.HMAlgebra
 |      minimalist_parser.algebras.algebra.Algebra
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name=None, zero=None, ops=None, syntax_op_names=None, meta=None)
 |      Initialise a string-building algebra with Bare Tree algebra terms.
 

Algebra constants are zeroary operations. They have a name and a zeroary function, which is an element of the domain: here, a string.

In [8]:
the = AlgebraOp("the", "the")

# The algebra also has a default constant maker
puppy = string_algebra.constant_maker("puppy")
snuggled = string_algebra.constant_maker("snuggled")

# look at the vocabulary
vocab = [the, puppy, snuggled]
for algebra_op in vocab:
    print("\nname:", algebra_op.name)
    print("function:", algebra_op.function)



name: the
function: the

name: puppy
function: puppy

name: snuggled
function: snuggled


Binary operations < and > can be looked up in `string_algebra.ops` or they can be created. 

For consistency across Head Movement Algebras, < is in `string_algebra.ops['concat_right']` and > is in `string_algebra.ops['concat_left']`. Notice both use the string_algebra.concat_right method, since Bare Trees are WYSIWYG when it comes to word order.

In [9]:
print("name:", string_algebra.ops['concat_right'].name)
print("function:", string_algebra.ops['concat_right'].function)
print("\nname:", string_algebra.ops['concat_left'].name)
print("function:", string_algebra.ops['concat_left'].function)

name: <
function: <bound method BareTreeStringAlgebra.concat_right of String Algebra>

name: >
function: <bound method BareTreeStringAlgebra.concat_right of String Algebra>


You can always just create an `AlgebraOp` as well, if you want.

In [10]:
new_right = AlgebraOp("<", string_algebra.concat_right)
print(new_right == string_algebra.ops['concat_right'])

True


### Algebra Terms

An term over an algebra is a tree in which the nodes are labelled with operations of that algebra, and the number of children of the node matches the arity of the function of the operation.

This means the leaves are constants, and the internal nodes are operations. 

An `AlgebraTerm` has a `parent`, which is an `AlgebraOp`, and, optionally, a list of `children`, which are `AlgebraTerm`s.

In [11]:
# for readability, let's give < and > names
# Type: AlgebraOp
head_left = string_algebra.ops['concat_right']
head_right = string_algebra.ops['concat_left']

# we also really want our vocabulary to be AlgebraTerms. We can turn them all into AlgebraTerms, but instead let's make them with the convenience function make_leaf
# Type: AlgebraTerm
the = string_algebra.make_leaf("the")
puppy = string_algebra.make_leaf("puppy")
snuggled = string_algebra.make_leaf("snuggled")

the_puppy = AlgebraTerm(head_left, [the, puppy])

print(the_puppy)

('<', [('the'), ('puppy')])


`AlgebraTerm`s can be exported to `nltk.Tree`s, which allows us to visualise them.

In [12]:
the_puppy.to_nltk_tree().draw()

In [13]:
the_puppy_snuggled = AlgebraTerm(head_right, [the_puppy, snuggled])
the_puppy_snuggled.to_nltk_tree().draw()

## Minimalist Algebras

A Minimalist Algebra is also an algebra, but it's quite different from a string-building algebra. A Minimalist Algebra essentially handles all the Move-related work. Structure-building is delegated to its "inner algebra", which can be, for instance, this `string_algebra`.

The domain of a Minimalist Algebra is `Expression`s, which contain an `inner_term` (a term of the inner algebra), `Movers`, and have an `mg_type` with things like +/- lexical and +/- conjunction.

`Movers` implements a partial function from slot names (such as `'-wh'` or `'Abar'`) to inner terms. 

Today we'll just look at the `MinimalistAlgebraSynchronous` subclass, so I can show you how to build a synchronous grammar over multiple inner algebras.

In [14]:
# a MinimalistAlgebraSynchronous requires a list of inner algebras.
minimalist_algebra = MinimalistAlgebraSynchronous([string_algebra])

help(minimalist_algebra)

Help on MinimalistAlgebraSynchronous in module minimalist_parser.minimalism.minimalist_algebra_synchronous object:

class MinimalistAlgebraSynchronous(minimalist_parser.minimalism.minimalist_algebra.MinimalistAlgebra)
 |  MinimalistAlgebraSynchronous(inner_algebras: list[minimalist_parser.algebras.algebra.Algebra], mover_type=<class 'minimalist_parser.minimalism.movers.Movers'>, prepare_packages=None, slots=None)
 |  
 |  A synchronous Minimalist Algebra: i.e. this has multiple Inner Algebras for structure-building.
 |  Attributes:
 |      inner_algebras: dict {Algebra: PreparePackages}
 |      inner_algebra: for compatability with parent class,
 |          the first algebra in the input parameter inner_algebras is stored here as a default inner algebra.
 |  
 |  Method resolution order:
 |      MinimalistAlgebraSynchronous
 |      minimalist_parser.minimalism.minimalist_algebra.MinimalistAlgebra
 |      minimalist_parser.algebras.algebra.Algebra
 |      builtins.object
 |  
 |  Method

In [15]:
print(minimalist_algebra.ops)

{}


In [16]:
minimalist_algebra.add_default_operations()

In [17]:
print(minimalist_algebra.ops)

{'Merge1_right': Merge1_right, 'Merge1_left': Merge1_left, 'Merge2_A': Merge2_A, 'Move1_right_A': Move1_right_A, 'Move1_left_A': Move1_left_A, 'Move2_A_A': Move2_A_A, 'Move2_A_ABar': Move2_A_ABar, 'Move2_A_R': Move2_A_R, 'Move2_A_Self': Move2_A_Self, 'Merge2_ABar': Merge2_ABar, 'Move1_right_ABar': Move1_right_ABar, 'Move1_left_ABar': Move1_left_ABar, 'Move2_ABar_A': Move2_ABar_A, 'Move2_ABar_ABar': Move2_ABar_ABar, 'Move2_ABar_R': Move2_ABar_R, 'Move2_ABar_Self': Move2_ABar_Self, 'Merge2_R': Merge2_R, 'Move1_right_R': Move1_right_R, 'Move1_left_R': Move1_left_R, 'Move2_R_A': Move2_R_A, 'Move2_R_ABar': Move2_R_ABar, 'Move2_R_R': Move2_R_R, 'Move2_R_Self': Move2_R_Self, 'Merge2_Self': Merge2_Self, 'Move1_right_Self': Move1_right_Self, 'Move1_left_Self': Move1_left_Self, 'Move2_Self_A': Move2_Self_A, 'Move2_Self_ABar': Move2_Self_ABar, 'Move2_Self_R': Move2_Self_R, 'Move2_Self_Self': Move2_Self_Self}


## Adding your own inner algebra

To do this, implement an `Algebra`. The `HMAlgebra` class has a bunch of default functions, so if you want an algebra with head movement, you may find this convenient. The `concat_right` etc methods by default just return `arg[0] + arg[1]` (or vice versa), so if you have a class you want to build, you may be able to get away with just initialising an `HMAlgebra` with `domain_type` specified.

In the original Chomsky 1995, he builds multisets of multisets. Python won't let you do that, so I implemented `FakeSet`, which inherits from `list`, but ignores order.

With this, you don't need to write a new class of `HMAlgebra`, just initialise one with `domain_type=FakeSet`.

In [18]:
from minimalist_parser.algebras.algebra_objects.fake_set import FakeSet

set_algebra = HMAlgebra("set algebra", FakeSet)

`HMAlgebra`s don't by default have a constant maker. We need to write our own, since a constant (lexical item) should already be a `FakeSet`, not just a string.

In [19]:
def make_fake_set_constant(word):
    """
    make an AlgebraOp with the given word as the content of a unary FakeSet.
    With this we can use the synchronous algebra make_leaf function for a shortcut to a term leaf.
    """
    return AlgebraOp(word, FakeSet([word]))

set_algebra.add_constant_maker(make_fake_set_constant)

In [20]:
# this is just the built-in concat_right method of HMAlgebras, which just uses + on the two arguments.
print(set_algebra.concat_right([FakeSet([7,8,1]), FakeSet([3, 2])]).spellout())

{{1, 7, 8}, {2, 3}}


In [21]:
# a term
# set_algebra.ops["concat_right"] is an AlgebraOp with name 'concat_right' and function set_algebra.concat_right.
t = AlgebraTerm(set_algebra.ops["concat_right"], [set_algebra.make_leaf("MG+"), AlgebraTerm(set_algebra.ops["concat_right"], [set_algebra.make_leaf("hi"), set_algebra.make_leaf("there")])])
t.to_nltk_tree().draw()
t.evaluate()


{{MG+}, {{hi}, {there}}}

In [22]:
# ignore these for now, they're just to make our lives easier later.
string_prepare = PreparePackagesBareTrees()
set_prepare = PreparePackagesHM("set prepare", set_algebra)


In [23]:
# ignore the prepare_packages part. They're not necessary, but they save us re-making examples later.
minimalist_algebra = MinimalistAlgebraSynchronous([string_algebra, set_algebra], prepare_packages=[string_prepare, set_prepare])

In [24]:
# Since all the inner algebras are HMAlgebras, there's a method for just adding all possible minimalist operations, right and left, given the movers slots.
minimalist_algebra.add_default_operations()

In [25]:
for op in minimalist_algebra.ops:
    print(op)

Merge1_right
Merge1_left
Merge2_A
Move1_right_A
Move1_left_A
Move2_A_A
Move2_A_ABar
Move2_A_R
Move2_A_Self
Merge2_ABar
Move1_right_ABar
Move1_left_ABar
Move2_ABar_A
Move2_ABar_ABar
Move2_ABar_R
Move2_ABar_Self
Merge2_R
Move1_right_R
Move1_left_R
Move2_R_A
Move2_R_ABar
Move2_R_R
Move2_R_Self
Merge2_Self
Move1_right_Self
Move1_left_Self
Move2_Self_A
Move2_Self_ABar
Move2_Self_R
Move2_Self_Self


In [26]:
# for readability, let's get some operations
merge_right = minimalist_algebra.ops["Merge1_right"]
merge_A = minimalist_algebra.ops["Merge2_A"]
move_A = minimalist_algebra.ops["Move1_left_A"]
merge_Abar = minimalist_algebra.ops["Merge2_ABar"]
move_Abar = minimalist_algebra.ops["Move1_left_ABar"]
move_A_Abar = minimalist_algebra.ops["Move2_A_ABar"]

In [27]:
# and some constants
the = minimalist_algebra.make_leaf("the")
puppy = minimalist_algebra.make_leaf("puppy")
snuggled = minimalist_algebra.make_leaf("snuggled")


If you don't want to use the default constant maker, you can pass `make_leaf` a dict from inner algebra to `InnerAlgebraInstructions`.

For example, if you need silent heads:

In [28]:
# we can usually just make a silent thing with the constructor of the inner domain.
# for example, str() makes "" and FakeSet() makes {}
# inner_algebra.domain_type() works in that case, but inner_algebra.empty_leaf_operation should work for any Algebra
silent_inners = {inner_algebra: InnerAlgebraInstructions(algebra_op=inner_algebra.empty_leaf_operation) for inner_algebra in minimalist_algebra.inner_algebras}
past = minimalist_algebra.make_leaf("[past]", silent_inners)

# see the silent heads:
print(set_algebra, past.parent, past.spellout(set_algebra))
print(string_algebra, past.parent, past.spellout(string_algebra))


set algebra [past] {}
String Algebra [past] 


In [29]:
# A term
t=SynchronousTerm(move_A, [SynchronousTerm(merge_right, [past, SynchronousTerm(merge_A, [snuggled, SynchronousTerm(merge_right, [the, puppy])])])])
t.to_nltk_tree().draw()

# See the inner terms
t.interp(set_algebra).inner_term.to_nltk_tree().draw()
t.interp(string_algebra).inner_term.to_nltk_tree().draw()



`spellout` is a shortcut for `t.interp(algebra).inner_term.evaluate()`. If the `domain_type` also has a `spellout` method, it also applies that, so e.g. a tree could spellout to its string yield or a `(string, triple, "")` could spell out to `"string triple"`.


In [30]:
print(t.spellout(set_algebra))
print(t.spellout(string_algebra))

{{{}, {snuggled}}, {{puppy}, {the}}}
the puppy snuggled


In [31]:
# Move2
who = minimalist_algebra.make_leaf("who")
q = minimalist_algebra.make_leaf("[Q]", silent_inners)


t2 = SynchronousTerm(move_Abar, [SynchronousTerm(merge_right, [q, SynchronousTerm(move_A_Abar, [SynchronousTerm(merge_right, [past, SynchronousTerm(merge_A, [snuggled, who])])])])])

# Minimalist term
t2.to_nltk_tree().draw()

# inner terms
t2.interp(set_algebra).inner_term.to_nltk_tree().draw()
t2.interp(string_algebra).inner_term.to_nltk_tree().draw()

# spellout
print(t2.spellout(set_algebra))
print(t2.spellout(string_algebra))



{{who}, {{}, {{}, {snuggled}}}}
who snuggled


## Prepare Packages

So far, we've had no Head Movement and moved items are not marked with traces. These kinds of things require* tree homomorphisms on the inner algebra terms.

These are called `PreparePackages`, and are paired with the `Algebra`s. `PreparePackagesHM` have methods to extract and combine heads.

* "require" is too strong a word. They can be built into the algebras if you want, but this misses generalisations, and they'll look different on the inner terms.


In [32]:
# for the string algebra, these are special since we need to follow with <, > arrows to find the head.
# by default, we just go down the left branches.
string_prepare = PreparePackagesBareTrees()
set_prepare = PreparePackagesHM("set prepare", set_algebra)


In [33]:
# e.g. extract_head returns a pair of the tree without its head and the head
print("sets")
remainder, head = set_prepare.extract_head(t2.interp(set_algebra).inner_term)
remainder.to_nltk_tree().draw()
print(head)

# removing the head in the string algebra leaves a trace t
print("strings")
remainder, head = string_prepare.extract_head(t2.interp(string_algebra).inner_term)
remainder.to_nltk_tree().draw()
print(head)

sets
('[empty]')
strings
('<e>')


A prepare package is a pair of tree homomorphisms that change, finitely, the functor term and the other term. ("other" = mover, or selectee, or modified)

In an HM algebra, these are just for head movement. As part of a minimalist operation, the output of the prepare package will then be combined with an operation of the inner algebra, such as `concat_right`.

The standard Prepare Packages for head movement algebras are:

* `suffix`: functor, other -> (functor with head = h_functor + other_functor , other without its head)
* `prefix`: functor, other -> (functor with head = other_functor h_functor , other without its head)
* `excorporation`: functor, other -> (other head , concat_right(functor head, other without its head)
* `hm_atb`: ONLY if functor head == other head:
  * functor, other -> (functor head, concat_right(functor without its head, other without its head)

A `MiniimalistFunction` is a type of `AlgebraOp` with a very complex constructor, taking things like the inner algebra operation, the prepare package, if any, and the slot(s) for movement.

For synchronous algebras, a `MinimalistFunctionSynchronous`, the inner-algebra-specific information is gathered in a dict from algebra to `InnerAlgebraInstructions`

In [34]:
help(InnerAlgebraInstructions)

Help on class InnerAlgebraInstructions in module minimalist_parser.minimalism.minimalist_algebra_synchronous:

class InnerAlgebraInstructions(builtins.object)
 |  InnerAlgebraInstructions(op_name: str = None, prepare: str = None, leaf_object=None, reverse=False, algebra_op=None)
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, op_name: str = None, prepare: str = None, leaf_object=None, reverse=False, algebra_op=None)
 |      Stores the necessary components for creating a MinimalistFunction for the particular inner algebra.
 |      @param op_name: the name of the inner op (to look it up in alg.ops or to give to alg.constant_maker.
 |      @param prepare: the name of the prepare operation, if any, for Merge1 or Move1.
 |      @param leaf_object: If this is a leaf, you can explicitly give the algebra object,
 |                          which will be used in place of whatever the default constant maker will create.
 |      @par

In [35]:
# since both of our algebras are HMAlgebras with PreparePackagesHM prepare packages, we can make in inner_ops dict very easily.
# for example, we can create a new minimalist operation with head-raising to prefix position.
prefix = {a: InnerAlgebraInstructions(op_name="concat_right", prepare="prefix") for a in minimalist_algebra.inner_algebras}

merge_right_prefix = MinimalistFunctionSynchronous(minimalist_algebra, minimalist_algebra.merge1, inner_ops=prefix, name="Merge1_right_prefix")

# we can add it to the algebra if we want.
minimalist_algebra.add_op(merge_right_prefix)

In [36]:
print(merge_right_prefix)

Merge1_right_prefix


In [37]:
ed = minimalist_algebra.make_leaf("-ed")
walk = minimalist_algebra.make_leaf("walk")

walked = SynchronousTerm(merge_right_prefix, [ed, walk])
walked.to_nltk_tree().draw()

# shortcut for walked.interp(string_algebra).inner_term.to_nltk_tree().draw()
walked.view_inner_term(string_algebra)
walked.spellout(string_algebra)

'walk -ed'

## Usage: we can make parse items without actually having to parse a sentence





In [11]:
from minimalist_parser.examples import tree_atb, cat_tried_sleep, tree_atb_hm, mg, triple_alg, am_alg


In [2]:
terms = tree_atb, cat_tried_sleep, tree_atb_hm
for term in terms:
    print(term.spellout(triple_alg))
    term.to_nltk_tree().draw()
    term.view_inner_term(triple_alg)
    

the cat dreamt and slept
the cat tried to sleep
did the dog dream and the cat sleep


In [3]:
from minimalist_parser.convert_mgbank.term2actions import add_interval_algebra
from minimalist_parser.minimalism.prepare_packages.addressed_triple_prepare_package import \
    HMAddressedTriplesPreparePackages
from minimalist_parser.algebras.hm_triple_algebra import HMTripleAlgebra
from minimalist_parser.minimalism.prepare_packages.interval_prepare_package import IntervalPairPrepare
from minimalist_parser.algebras.hm_interval_pair_algebra import HMIntervalPairsAlgebra
# make required algebras and add to MG


In [4]:
# IntervalPairs implement pairs of intervals a la Milos's paper on parsing complexity with head movement
# (but my version)
# (head interval, rest of the phrase interval) + typing to tell you what operations can apply (must_hm and lexical)
example_interval_algebra = HMIntervalPairsAlgebra()
example_interval_prepare = IntervalPairPrepare()
example_interval_algebra.add_constant_maker()

# interpret into MG tree addresses, to track where words in the sentence came from (including deletion due to ATB movement)
example_address_algebra = HMTripleAlgebra("addresses algebra", component_type=list, addresses=True)
example_address_prepare = HMAddressedTriplesPreparePackages()

mg.inner_algebras[example_address_algebra] = example_address_prepare
mg.inner_algebras[example_interval_algebra] = example_interval_prepare

for term in [tree_atb, cat_tried_sleep, tree_atb_hm]:
    add_interval_algebra(term, mg, triple_alg, example_address_algebra, example_address_prepare,
                         example_interval_algebra, example_interval_prepare, False)


In [5]:
print(tree_atb.spellout(example_address_algebra))
print(tree_atb.spellout(triple_alg))
print(tree_atb.spellout(example_interval_algebra))
tree_atb.to_nltk_tree().draw()

[['01110', '010110'], ['01111', '010111'], ['0110'], ['0100'], ['01010']]
the cat dreamt and slept
(0,5)


In [6]:
tree_atb.view_inner_term(example_interval_algebra)

In [10]:
for alg in mg.inner_algebras:
    print()
    print(alg)
    print(tree_atb.spellout(alg))


string_algebra
the cat dreamt and slept

AM algebra
rt:			0
nodes:		{0, 1, 2, 3}
labels:		{3: 'poes', 1: 'dream-01', 2: 'sleep-01', 0: 'and'}
sources:	{}
edges:
	 1 ARG0 3
	 2 ARG0 3
	 0 op1 1
	 0 op2 2


string triples
the cat dreamt and slept

addresses algebra
[['01110', '010110'], ['01111', '010111'], ['0110'], ['0100'], ['01010']]

Interval Pair Algebra
(0,5)


In [12]:
g = tree_atb.spellout(am_alg)

g.to_graphviz()

AttributeError: 'str' object has no attribute 'to_graphviz'